In [2]:
#CSV Bibliothek importieren 
import csv
#Pandas importieren und als pd bezeichnen
import pandas as pd
#Numpy importieren und als np bezeichnen 
import numpy as np
#Json Biliothek importieren
import json
#Obere Schranke für die Größe der Daten auf 100,000,000 setzen  
csv.field_size_limit(100000000)
#Maximale Anzahl von Zeilen und Spalten Zeigen 
pd.options.display.max_columns = None
pd.options.display.max_rows = None
#Imortiert die Bibliothel tldextract, um die Domaine zu extrahieren
import tldextract
#Diese Biliothek berechnet die Lesbarkeit
from fkscore import fkscore
#RegEx-bibliothek importieren
import re

----

# Anzahl der Hashtags, Erwähnungen, URLs sowie Lesbarkeit und Textlänge
#### Zuerst werden die Tweets in TweetsCov19 bearbeitet, dann die Tweets in TweetsKB

****

## TweetsCov19:

1. Die **Anzahl der Erwähnungen** werden berechnet
2. Die **Anzahl der Hashtags** werden berechnet
3. Die **Anzahl der URLs** werden berechnet
4. Die **Domäne** aus URLs extrahiert
5. Die **Textlänge** berechnet
6. Die neue Features werden gespeichert

***

In [6]:
#Labels und Pfad für den TweetsCov19
TweetsCov19_labels = ['Tweet_Id', "Benutzername", 'Zeitstempel', 'Abonnenten', 'Abonnierten', 'Retweets', 'Likes', 'Entities', 'Stimmung', 'Erwähnungen', 'Hashtags', 'URLs', 'Text', 'Ist_gelöscht']
TweetsCov19_Pfad =  'C:/Mo/tweetDB/tweetscov19/TweetsCov19_tsv/data_sample/TweetsCov19_stichprobe.tsv'

In [8]:
#Stichprobe von Tweetscov19 lesen

df_tweetscov19 = pd.read_csv(TweetsCov19_Pfad, sep='\t', error_bad_lines=False, names=TweetsCov19_labels)
print('Form', df_tweetscov19.shape)
df_tweetscov19.head()

Form (9409841, 14)


,Tweet_Id,Benutzername,Zeitstempel,Abonnenten,Abonnierten,Retweets,Likes,Entities,Stimmung,Erwähnungen,Hashtags,URLs,Text,Ist_gelöscht
0,1178791637348212736,c57d4fb895c5f4a33d2a2b24709f93b5,2019-09-30 22:00:01+00:00,764191,132,19,180,foodie:Foodie:-2.096663959988218;,1 -1,null;,HalloweenTime?,null;,Has your appetite materialized just as fast as...,True
1,1178791641601232896,627f7d4ef584ba33a676a4ebcb861cdb,2019-09-30 22:00:02+00:00,321189,26833,32,43,mainland china:Mainland_China:-1.6252666161640...,1 -1,null;,HongKong,https://www.catholicnewsagency.com/news/amid-h...,The persecution of Catholics in mainland China...,False
2,1178791643509612547,7966d5716c1c4f33591b17755184e3d6,2019-09-30 22:00:03+00:00,14209,10623,90,182,brexit:Brexit:-1.582809637810362;hear hear:Hea...,1 -2,null;,Brexit,null;,"Hear, hear..\nEmily: You were an outspoken cri...",False
3,1178791644214235137,f17996c036ed4cd981f5448fe00d9ed2,2019-09-30 22:00:03+00:00,2264986,655,60,528,null;,3 -1,amstandardca.,BlueJays,null;,We wrapped up the season with a WINNING week! ...,False
4,1178791645371875329,32a54e5f6d8a67de1bffd2e689594dc8,2019-09-30 22:00:03+00:00,171,138,33,117,my family:My_Family:-2.6353832182503094;roll t...,4 -1,null;,null;,null;,Super excited to announce that I will be spend...,False


In [9]:
#Diese Funktion bekommt eine String, wandelt die String-Daten zu einer Liste von URLs um. Dann extrahieret die Domäne aus der URLs.

def get_domain(url):
    s = url.split(':-:')
    f = filter(None, s)
    l2 = list(f)
    domain_txt = ''
    for x in range(0, len(l2)):
        ex = tldextract.extract(l2[x])
        if x != 0:
            domain_txt = domain_txt + ' ' + ex.domain
        else:
            domain_txt = domain_txt + ex.domain
    return domain_txt


In [10]:
#Zuerst wird die Spalte Erwähnungen als String definiert, dann werden String-Daten zu einer Liste umgewandelt. 
#Die Länge der Liste wird als die Anzahl der Erwähnungen berechnet und zurückgegeben

df_tweetscov19['Erwähnungen'] = df_tweetscov19['Erwähnungen'].astype(str)
df_tweetscov19['Anzahl der Erwähnungen'] = df_tweetscov19.apply(lambda row: len(list(row['Erwähnungen'].split())) if (row['Erwähnungen'] != 'null;') else 0, axis=1)
print('Anzahl der Erwähnungen wurde berechnet!')

Anzahl der Erwähnungen wurde berechnet!


In [11]:
#Zuerst wird die Spalte Hashtags als String definiert, dann werden String-Daten zu einer Liste umgewandelt. 
#Die Länge der Liste wird als die Anzahl der Hashtags berechnet und zurückgegeben

df_tweetscov19['Hashtags'] = df_tweetscov19['Hashtags'].astype(str)
df_tweetscov19['Anzahl der Hashtags'] = df_tweetscov19.apply(lambda row: len(list(row['Hashtags'].split())) if (row['Hashtags'] != 'null;') else 0, axis=1)
print('Anzahl der Hashtags wurde berechnet!')

Anzahl der Hashtags wurde berechnet!


In [12]:
#Zuerst wird die Spalte URLs als String definiert, dann werden String-Daten zu einer Liste umgewandelt. 
#Die Liste wird dann gefilterd, um die leere Teile zu entfernen. Dann wird die Länge der Liste wird als die Anzahl der URLs berechnet und zurückgegeben

df_tweetscov19['URLs'] = df_tweetscov19['URLs'].astype(str)
df_tweetscov19['Anzahl der URLs'] = df_tweetscov19.apply(lambda row: len(list(filter(None,(row['URLs'].split(':-:'))))) if (row['URLs'] != 'null;') else 0, axis=1)
print('Anzahl der URLs wurde berechnet!')

Anzahl der URLs wurde berechnet!


In [13]:
#Ruft diese Methode die Funktion get_domain(). Die Funktion liest jede einzelne Urls von dem String, extrahiert die Domain davon.
#Alle Domaine werden mit einer Leerzeichen voneinander getrennt, gespeichert und zurückgegeben.

df_tweetscov19['Domäne'] = df_tweetscov19.apply(lambda row: get_domain(row['URLs']) if (row['URLs'] != 'null;') else 0, axis=1)
print('Domäne wurden extrahiert!')

Domäne wurden extrahiert!


In [14]:
#Textlänge wird als Anzahl von benutzten Zeichen berechnet und gespeichert.

df_tweetscov19['Text'] = df_tweetscov19['Text'].astype(str)
df_tweetscov19['Textlänge'] = df_tweetscov19.apply(lambda row: len(row['Text']) , axis=1)
print('Textlänge wurde berechnet!')

Textlänge wurde berechnet!


In [15]:
#Diese Funktion berechnet die Lesbarkeit anhand der fkscore_bibliothek und gibt das Ergebnis zurück.

def get_fkscore_readablity(text):
    fk = fkscore(text)
    return fk.score['readability']

In [16]:
#Die Funktion ruf get_fkscore_readablity(), und das zurückgegebene Ergebnis speichert.

df_tweetscov19['Lesbarkeit'] = df_tweetscov19.apply(lambda row: get_fkscore_readablity(row['Text']), axis=1)
print('Lesbarkeit wurde berechnet!')

Lesbarkeit wurde berechnet!


In [17]:
df_tweetscov19.head()

,Tweet_Id,Benutzername,Zeitstempel,Abonnenten,Abonnierten,Retweets,Likes,Entities,Stimmung,Erwähnungen,Hashtags,URLs,Text,Ist_gelöscht,Anzahl der Erwähnungen,Anzahl der Hashtags,Anzahl der URLs,Domäne,Textlänge,Lesbarkeit
0,1178791637348212736,c57d4fb895c5f4a33d2a2b24709f93b5,2019-09-30 22:00:01+00:00,764191,132,19,180,foodie:Foodie:-2.096663959988218;,1 -1,null;,HalloweenTime?,null;,Has your appetite materialized just as fast as...,True,0,1,0,0,155,76.553
1,1178791641601232896,627f7d4ef584ba33a676a4ebcb861cdb,2019-09-30 22:00:02+00:00,321189,26833,32,43,mainland china:Mainland_China:-1.6252666161640...,1 -1,null;,HongKong,https://www.catholicnewsagency.com/news/amid-h...,The persecution of Catholics in mainland China...,False,0,1,1,catholicnewsagency,204,29.468
2,1178791643509612547,7966d5716c1c4f33591b17755184e3d6,2019-09-30 22:00:03+00:00,14209,10623,90,182,brexit:Brexit:-1.582809637810362;hear hear:Hea...,1 -2,null;,Brexit,null;,"Hear, hear..\nEmily: You were an outspoken cri...",False,0,1,0,0,163,79.534
3,1178791644214235137,f17996c036ed4cd981f5448fe00d9ed2,2019-09-30 22:00:03+00:00,2264986,655,60,528,null;,3 -1,amstandardca.,BlueJays,null;,We wrapped up the season with a WINNING week! ...,False,1,1,0,0,132,85.889
4,1178791645371875329,32a54e5f6d8a67de1bffd2e689594dc8,2019-09-30 22:00:03+00:00,171,138,33,117,my family:My_Family:-2.6353832182503094;roll t...,4 -1,null;,null;,null;,Super excited to announce that I will be spend...,False,0,0,0,0,257,61.203


In [16]:
#Die Ergebnisse werden gespeichert
TweetsCov19_Pfad =  'C:/Mo/tweetDB/tweetscov19/TweetsCov19_tsv/data_sample/TweetsKB_stichprobe_lesbarkeit.tsv'

with open(TweetsCov19_Pfad, 'w', newline='', encoding="utf-8") as out_file:
    tsv_writer = csv.writer(out_file, delimiter="\t")
    for idx, row in df_tweetscov19.iterrows():
        tsv_writer.writerow(row.values.tolist())

***

# TweetsKB:

1. Die **Hashtags** aus dem Text werden extrahiert
2. Die **Erwähnungen** aus dem Text werden extrahiert
3. Die **Anzahl der Hashtags** werden berechenet
4. Die **Anzahl der Erwähnungen** werden berechnet
5. Die **Textlänge** wird berechnet
6. Die neue Features werden gespeichert

***

In [2]:
#Labels und Pfad für den TweetsKB

TweetsKB_labels    = ['Tweet Id', 'Benutzername', 'Zeitstempel', 'Abonnenten', 'Abonnierten', 'Retweets', 'Likes', 'Text', 'URLs', 'Ist_gelöscht' ]
TweetsKB_Pfad =  'C:/Mo/tweetDB/tweetscov19/TweetsCov19_tsv/data_sample/TweetsKB_stichprobe.tsv'

In [3]:
#TweetsKB wird gelesen

df_tweetskb = pd.read_csv(TweetsKB_Pfad, sep='\t', error_bad_lines=False, names=TweetsKB_labels)
print('Form :', df_tweetskb.shape)
df_tweetskb.head()

Form : (9409841, 10)


,Tweet Id,Benutzername,Zeitstempel,Abonnenten,Abonnierten,Retweets,Likes,Text,URLs,Ist_gelöscht
0,1178791636039589889,DCComics,2019-09-30 22:00:01+00:00,3349606,563,2,13,"Today on #DCDaily, the crew takes a deep dive ...",http://bit.ly/2mlNYpS:-:,False
1,1178791641265639424,seanberdyz,2019-09-30 22:00:02+00:00,1203,499,4,20,𝘪’𝘮 𝘨𝘰𝘯𝘯𝘢 𝘣𝘦 𝘳𝘪𝘨𝘩𝘵 𝘩𝘦𝘳𝘦 ⟶ https://t.co/kOxWQL5...,http://ourownside.carrd.co:-:,True
2,1178791640573579264,AnArtistAtBirth,2019-09-30 22:00:02+00:00,109,215,0,0,❛ 𝘠𝘰𝘶'𝘳𝘦 𝘴𝘪𝘵𝘵𝘪𝘯' 𝘰𝘯 𝘺𝘰𝘶𝘳 𝘧𝘦𝘦𝘭𝘪𝘯𝘨𝘴 . . . 𝘐'𝘮 𝘴𝘪...,null;,True
3,1178791644797255680,lagosboygang,2019-09-30 22:00:03+00:00,571,1401,0,0,@Tiffany_o_ @UG_derrick That's the one i wanna...,null;,True
4,1178791648056217600,Phenom_Hoops,2019-09-30 22:00:04+00:00,25630,1674,1,1,2022 6’9 Perry Smith Jr. showcasing his high-m...,https://www.phenomhoopreport.com/2022-69-perry...,False


In [5]:
#Diese Funktion extrahiert die Hashtags anhand der Reguläre Ausdrücke aus dem Text und gibt sie Züruck.
def get_hashtags(text):
    return re.findall("(?<![@\w])#(\w{1,25})", text)

#Die Funktion get_Hashtags wird gerufen, zurückgegebene Hashtags werden in der Spalte Hashtags gespeichert.

df_tweetskb['Text'] = df_tweetskb['Text'].astype(str)
df_tweetskb['Hashtags'] = df_tweetskb.apply(lambda row: get_hashtags(row['Text']) , axis=1)
print('Hashtags wurden extrahiert!')

Hashtags wurden extrahiert!


In [6]:
#Diese Funktion extrahiert die Erwähnungen anhand der Reguläre Ausdrücke aus dem Text und gibt sie in einer Liste Züruck.

def get_mentions(text):
    return re.findall("(?<![@\w])@(\w{1,25})", text)

#Die Funktion get_mentions wird gerufen, zurückgegebene Mentions werden in der Spalte Mentions gespeichert.

df_tweetskb['Text'] = df_tweetskb['Text'].astype(str)
df_tweetskb['Mentions'] = df_tweetskb.apply(lambda row: get_mentions(row['Text']) , axis=1)
print('Mentions wurden extrahiert!')

Mentions wurden extrahiert!


In [15]:
import ast

#Hier werden die String Daten zuerst zu einer Liste umgewandelt. Dann die Länge der Liste als die Anzahl der Hashtags zurückgegeben.

def get_hashtags_number(hashtags):
    #t = ast.literal_eval(hashtags)
    return len(hashtags)

df_tweetskb['Anzahl der Hashtags'] = df_tweetskb.apply(lambda row: get_hashtags_number(row['Hashtags']) , axis=1)
print('Anzahl der Hashtags wurde berechnet!')

Anzahl der Hashtags wurde berechnet!


In [27]:
#Hier werden die String Daten zuerst zu einer Liste umgewandelt. Dann die Länge der Liste als die Anzahl der Mentions zurückgegeben.

def get_mentions_number(mentions):
    #t = ast.literal_eval(mentions)
    return len(mentions)

df_tweetskb['Anzahl der Erwähnungen'] = df_tweetskb.apply(lambda row: get_mentions_number(row['Mentions']) , axis=1)
print('Anzahl der Mentions wurde berechnet!')

Anzahl der Mentions wurde berechnet!


In [18]:
#Textlänge wird als Anzahl von benutzten Zeichen berechnet und gespeichert.

df_tweetskb['Text'] = df_tweetskb['Text'].astype(str)
df_tweetskb['Textlänge'] = df_tweetskb.apply(lambda row: len(row['Text']) , axis=1)
print('Textlänge wurde berechnet!')

Textlänge wurde berechnet!


In [24]:
#Die Funktion get_fkscore_readablity() wird gerufen, das zurückgegebene Ergebnis wird gespeichert.

df_tweetskb['Lesbarkeit'] = df_tweetskb.apply(lambda row: get_fkscore_readablity(row['Text']), axis=1)
print('Lesbarkeit wurde berechnet!')

Lesbarkeit wurde berechnet!


In [10]:
#Die erste 5 Zeile ausgeben
df_tweetskb.head()

,Tweet_Id,Benutzername,Zeitstempel,Abonnenten,Abonnierten,Retweets,Likes,Text,URLs,Ist_gelöscht,Hashtags,Erwähnungen,Anzahl der Hashtags,Anzahl der Mentions,Textlänge,Lesbarkeit
0,1178791636039589889,DCComics,2019-09-30 22:00:01+00:00,3349606,563,2,13,"Today on #DCDaily, the crew takes a deep dive ...",http://bit.ly/2mlNYpS:-:,False,['DCDaily'],['DCUTitans'],1,1,192,70.350
1,1178791641265639424,seanberdyz,2019-09-30 22:00:02+00:00,1203,499,4,20,𝘪’𝘮 𝘨𝘰𝘯𝘯𝘢 𝘣𝘦 𝘳𝘪𝘨𝘩𝘵 𝘩𝘦𝘳𝘦 ⟶ https://t.co/kOxWQL5...,http://ourownside.carrd.co:-:,True,[],[],0,0,73,82.390
2,1178791640573579264,AnArtistAtBirth,2019-09-30 22:00:02+00:00,109,215,0,0,❛ 𝘠𝘰𝘶'𝘳𝘦 𝘴𝘪𝘵𝘵𝘪𝘯' 𝘰𝘯 𝘺𝘰𝘶𝘳 𝘧𝘦𝘦𝘭𝘪𝘯𝘨𝘴 . . . 𝘐'𝘮 𝘴𝘪...,null;,True,['𝘛𝘩𝘦𝘰𝘯𝘦'],[],1,0,128,123.755
3,1178791644797255680,lagosboygang,2019-09-30 22:00:03+00:00,571,1401,0,0,@Tiffany_o_ @UG_derrick That's the one i wanna...,null;,True,[],"['Tiffany_o_', 'UG_derrick']",0,2,77,105.921
4,1178791648056217600,Phenom_Hoops,2019-09-30 22:00:04+00:00,25630,1674,1,1,2022 6’9 Perry Smith Jr. showcasing his high-m...,https://www.phenomhoopreport.com/2022-69-perry...,False,['PhenomHoops'],[],1,0,227,65.943


In [29]:
#Dataframe wird hier gespeichert.

TweetsKB_Pfad =  'C:/Mo/tweetDB/tweetscov19/TweetsCov19_tsv/data_sample/TweetsKB_stichprobe_lesbarkeit.tsv'

with open(TweetsKB_Pfad, 'w', newline='', encoding="utf-8") as out_file:
    tsv_writer = csv.writer(out_file, delimiter="\t")
    for idx, row in df_tweetskb.iterrows():
        tsv_writer.writerow(row.values.tolist())